In [ ]:
import json
import os
from dataclasses import dataclass
from typing import Dict, List
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)

from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

In [ ]:
# ========== 配置 ==========
MODEL_NAME = "/model/HuggingFace/deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"  # 或者你在 HF 上的 chatglm3-6b 的 repo 名
YAOLAO_JSON = "/root/yaolao/train_data_filtered.json"
OUTPUT_DIR = "lora_DeepSeek-R1-Distill-Qwen-14B_yaolao"

In [ ]:
# 训练超参（按需调整）
NUM_EPOCHS = 3
LEARNING_RATE = 2e-4
PER_DEVICE_BATCH_SIZE = 2  # 取决于显存
GRAD_ACCUM_STEPS = 8
WEIGHT_DECAY = 0.0
LOGGING_STEPS = 50
SAVE_STEPS = 500
MAX_LENGTH = 1024  # 根据模型和显存调整
# LoRA 配置（按需调整）
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q_proj", "v_proj"]  # 常见于 transformer 层，chatglm 具体实现可能不同；如果无效可换成 ["query_key_value"] 等

In [ ]:
# ========== 辅助函数：把 conversations 拼成训练样本 ==========
def build_examples_from_conversations(conversations: List[Dict]) -> List[Dict]:
    """
    将你给出的 conversation list 转成多条训练样本：
    每一条样本的 input 是 system + (user + assistant)* 到 assistant 之前的上下文，
    label 仅为 assistant 的回复（user/system 部分 label = -100）。
    假定 conversations 内 role 顺序是 system/user/assistant/user/assistant...
    """
    examples = []
    # 把整个 conversation 对话按轮次拼接。我们把每一次 assistant 回复作为一个训练目标
    system_content = ""
    idx = 0
    # 先找 system，如果存在则记录
    if len(conversations) and conversations[0].get("role") == "system":
        system_content = conversations[0].get("content", "")
        idx = 1

    # 从 idx 开始遍历，构造上下文
    history: List[Dict] = []
    # 收集对话为一系列 (user, assistant)
    while idx < len(conversations):
        role = conversations[idx].get("role")
        if role == "user":
            user_content = conversations[idx].get("content", "")
            # 如果后面有 assistant，作为一轮
            if idx + 1 < len(conversations) and conversations[idx + 1].get("role") == "assistant":
                assistant_content = conversations[idx + 1].get("content", "")
                # 构造样本：上下文为 system + 历史轮 + 当前 user，label 为 assistant
                # 保存当前历史（包括之前的 user/assistant）
                rounds = history.copy()
                rounds.append({"user": user_content, "assistant": None})
                # build prompt text for input (we will append assistant target in labels handling)
                prompt = ""
                if system_content:
                    prompt += f"<|system|>:\n{system_content}\n"
                # 每轮用一个简单模板拼接（你可以根据 chatglm 期待的特殊标记修改）
                for r in rounds[:-1]:  # 历史完整轮（都有 assistant）
                    prompt += f"<|user|>:\n{r['user']}\n<|assistant|>:\n{r['assistant']}\n"
                # 最后一轮只加 user（assistant 部分用作 label）
                prompt += f"<|user|>:\n{user_content}\n<|assistant|>:\n"

                examples.append({
                    "input_text": prompt,
                    "target_text": assistant_content
                })

                # 更新历史：把这轮完整加入 history
                history.append({"user": user_content, "assistant": assistant_content})
                idx += 2
            else:
                # 不成对（没有 assistant），只保存 user 进历史
                history.append({"user": user_content, "assistant": None})
                idx += 1
        else:
            # 如果遇到 assistant 单独项或其他，直接跳过以防格式不规范
            idx += 1
    return examples

In [ ]:
# ========== 主流程 ==========
def main():
    # 1. 读取 json
    with open(YAOLAO_JSON, "r", encoding="utf-8") as f:
        raw = json.load(f)

    # raw 可能是 {"conversations":[{...},{...}], ...} 或者是一个列表，每条为一段对话
    # 适配常见两种情况
    examples_all = []
    if isinstance(raw, dict) and "conversations" in raw and isinstance(raw["conversations"], list):
        # 单条 conversation（整体）
        examples_all += build_examples_from_conversations(raw["conversations"])
    elif isinstance(raw, list):
        # 每个元素可能是一段 conversations
        for item in raw:
            if "conversations" in item:
                examples_all += build_examples_from_conversations(item["conversations"])
    else:
        raise ValueError("无法识别 yaolao.json 的结构，请确认根结构是 dict 且包含 conversations，或是 list。")

    print(f"构造了 {len(examples_all)} 个训练样本示例（每个样本对应一条 assistant 回复）")

    # 2. 加载 tokenizer + 模型（可按需加 trust_remote_code）
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    # chatglm 系列有些 tokenizer 需要设置 truncation_side etc.
    tokenizer.padding_side = "right"

    # 通过 bitsandbytes 低精度加载模型（如果没有 bitsandbytes 可去掉 load_in_8bit 参数）
    # 准备 model（k-bit friendly）
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        device_map="auto",
        load_in_8bit=True,   # 如果你不使用 bitsandbytes 删除或改为 False
        llm_int8_enable_fp32_cpu_offload=True
    )

    # 使模型为 k-bit/8bit 微调做好准备
    model = prepare_model_for_kbit_training(model)

    # 3. 应用 LoRA（PEFT）
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        lora_dropout=LORA_DROPOUT,
        target_modules=TARGET_MODULES,
    )
    model = get_peft_model(model, peft_config)
    print("已加载模型并应用 LoRA。可训练参数量（参数名/形状）：")
    # 打印可训练参数数量
    trainable_params = 0
    total_params = 0
    for n, p in model.named_parameters():
        num = p.numel()
        total_params += num
        if p.requires_grad:
            trainable_params += num
    print(f"Trainable params: {trainable_params} / Total params: {total_params}")

    # 4. 构造 Dataset（tokenize 并且 labels 仅包含 assistant）
    def tokenize_and_build_labels(example):
        # 把 input_text 和 target_text 合并： input_ids = tokenize(input_text + target_text)
        # labels = [-100]*len(input_ids(input_text)) + token_ids(target_text)
        # 这样 trainer 只会对 assistant 部分计算 loss
        input_text = example["input_text"]
        target_text = example["target_text"]

        # 直接拼接
        full_text = input_text + target_text
        # tokenize
        input_enc = tokenizer(
            full_text,
            max_length=MAX_LENGTH,
            truncation=True,
            padding=False,  # 由 data collator 负责 padding
        )
        input_ids = input_enc["input_ids"]

        # token length of the input_text part
        input_text_ids = tokenizer(input_text, add_special_tokens=False)["input_ids"]
        input_len = len(input_text_ids)

        labels = [-100] * input_len + input_ids[input_len:]
        labels = labels[:MAX_LENGTH]
        if len(input_ids) > MAX_LENGTH:
            input_ids = input_ids[:MAX_LENGTH]

        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": [1] * len(input_ids),
        }

    ds = Dataset.from_list(examples_all)
    ds_tokenized = ds.map(lambda x: tokenize_and_build_labels(x), remove_columns=ds.column_names, num_proc=1)

    # 5. Data collator（使用 transformers 的 DataCollatorForSeq2Seq 或 自定义）
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

    # 6. TrainingArguments & Trainer
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUM_STEPS,
        warmup_ratio=0.03,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=LOGGING_STEPS,
        save_strategy="steps",
        save_steps=SAVE_STEPS,
        save_total_limit=3,
        remove_unused_columns=False,
        report_to="none",  # 如果希望用 wandb 或 tensorboard，请配置
        optim="paged_adamw_8bit",  # 结合 bitsandbytes 推荐的优化器
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds_tokenized,
        data_collator=data_collator,
    )

    # 7. 开始训练
    trainer.train()

    # 8. 保存 LoRA 权重（只保存 adapter）
    model.save_pretrained(OUTPUT_DIR)
    print(f"训练完成并将 LoRA 权重保存到 {OUTPUT_DIR}")

    # 9. 如果需要，将最终模型推到 hub（可选）
    # from huggingface_hub import login
    # login(token="你的HfToken")
    # model.push_to_hub("your-username/lora-chatglm3-yaolao")

if __name__ == "__main__":
    main()